# Руководство: Подготовка данных осциллограмм с ProcessingOscillograms

Этот Jupyter Notebook демонстрирует последовательный процесс обработки и подготовки данных осциллограмм с использованием методов класса `ProcessingOscillograms` из модуля `preparing_oscillograms.processing_oscillograms`.

**Цель**: Показать, как шаг за шагом применять различные функции класса для очистки, стандартизации и анализа конфигурационных файлов осциллограмм (.cfg).

**Рассматриваемые шаги:**
1.  Удаление конфиденциальной информации и переименование файлов на основе хеша.
2.  Замена даты изменения файлов.
3.  Группировка файлов по частоте дискретизации и частоте сети.
4.  Сбор всех уникальных имен аналоговых и цифровых сигналов.
5.  Ручная стандартизация имен (описание процесса).
6.  Переименование аналоговых и цифровых сигналов на основе стандартизированных имен.
7.  Поиск дубликатов имен сигналов в файлах конфигурации.

**Важно**: Примеры в этом ноутбуке предполагают наличие файлов `.cfg` и соответствующих им `.dat` файлов. Для выполнения примеров кода вам может потребоваться создать демонстрационные файлы и каталоги, как указано в ячейках кода, или адаптировать пути к вашим существующим данным.

In [ ]:
import sys
import os

# Получаем текущий рабочий каталог ноутбука
notebook_dir = os.getcwd()

# Выход на исходный коталог
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir))

sys.path.append(project_root)
print(f"Добавлен путь в sys.path: {project_root}")

In [ ]:
import os
import shutil
import json
import datetime
import csv # Хотя сам ProcessingOscillograms его использует, здесь он может понадобиться для создания dummy csv

from preparing_oscillograms.processing_oscillograms import ProcessingOscillograms

# --- Инициализация класса ---
processor = ProcessingOscillograms()
print("Экземпляр класса ProcessingOscillograms создан.")

# --- Базовые пути для демонстрационных данных ---
# Эти пути будут использоваться в примерах ниже.
# Вы можете изменить их или создать эти каталоги и файлы вручную.

# Каталог для исходных .cfg файлов (до обработки)
# !!! ВАЖНО !!!
# Файлы в данной дирректории как раз будут меняться. Поэтому это должна быть копия для обработки.
# !!! ВАЖНО !!!
base_source_cfg_dir = os.path.join(project_root, "raw_data","002_run_processing_oscillograms_guide","")

os.makedirs(base_source_cfg_dir, exist_ok=True)

print("Начальная настройка завершена. Можно приступать к демонстрации методов.")

## 1. Удаление конфиденциальной информации и переименование файлов

Метод `deleting_confidential_information_in_all_files` выполняет две основные задачи:
1.  **Удаление локальной информации**: В каждой строке файла `.cfg`, содержащей информацию о станции и номере устройства (обычно первая строка), эта информация заменяется на пустые значения (например, `",,"`). Даты начала и конца записи также заменяются на `01/01/0001,01:01:01.000000`.
2.  **Переименование файлов**: После обработки файла `.cfg`, он и соответствующий ему файл `.dat` переименовываются на основе MD5-хеша содержимого файла `.dat`. Например, `original_name.cfg` и `original_name.dat` станут `[hash_value].cfg` и `[hash_value].dat`.

Этот шаг важен для анонимизации данных перед их дальнейшим использованием или публикацией.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` и `.dat` для обработки.

**Побочные эффекты:**
*   Изменяет содержимое файлов `.cfg` в `source_dir`.
*   Переименовывает файлы `.cfg` и `.dat` в `source_dir`.
*   Создает файл `protected_files.txt` в `source_dir`, содержащий список файлов, которые не удалось обработать (например, из-за проблем с кодировкой), и ошибки обработки.

In [ ]:
# --- 1. Демонстрация deleting_confidential_information_in_all_files ---

# Используем ранее определенный base_source_cfg_dir
print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Вызов метода
print("\nВызов deleting_confidential_information_in_all_files...")
processor.deleting_confidential_information_in_all_files(source_dir=base_source_cfg_dir)
print("Метод deleting_confidential_information_in_all_files завершен.")

## 2. Замена даты изменения файлов

Метод `date_of_change_replacement` обновляет системную дату последнего изменения для всех файлов в указанном каталоге. Каждому файлу присваивается текущая системная дата и время на момент выполнения этого метода.

Это может быть полезно для стандартизации метаданных файлов или для того, чтобы отметить этап обработки.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы, у которых нужно обновить дату изменения.

**Побочные эффекты:**
*   Изменяет системную дату последнего изменения файлов в `source_dir`.

In [ ]:




# --- ОСТАНОВИЛСЯ ТУТ!!!




# --- 2. Демонстрация date_of_change_replacement ---

# Для этого шага мы будем использовать файлы, обработанные на предыдущем этапе.
# Каталог processing_dir_step1 содержит файлы с именами-хешами.
# Если предыдущая ячейка не выполнялась, создадим каталог и несколько файлов.
if not 'processing_dir_step1' in globals() or not os.path.exists(processing_dir_step1) or not os.listdir(processing_dir_step1):
    print(f"Каталог 'processing_dir_step1' не существует или пуст. Создаем заново для этого шага.")
    # Если processing_dir_step1 не определен, установим его здесь, как в предыдущей ячейке
    if not 'processing_dir_step1' in globals():
        processing_dir_step1 = os.path.join(base_source_cfg_dir, "step1_delete_confidential")
    if os.path.exists(processing_dir_step1): # Если существует, но пуст
        shutil.rmtree(processing_dir_step1)
    os.makedirs(processing_dir_step1)
    create_dummy_cfg_dat_pair(processing_dir_step1, "hash1_dummy", content_cfg=",,REC\n1,1A,0D\nDummySignal\n50\n1\n1000,1\n01/01/0001,01:01:01.000000\n01/01/0001,01:01:01.000000\nDAT", content_dat="data1")
    create_dummy_cfg_dat_pair(processing_dir_step1, "hash2_dummy", content_cfg=",,REC\n1,1A,0D\nDummySignal2\n50\n1\n1000,1\n01/01/0001,01:01:01.000000\n01/01/0001,01:01:01.000000\nDAT", content_dat="data2")
    print(f"Созданы фиктивные файлы в '{processing_dir_step1}' для демонстрации date_of_change_replacement.")

print(f"Каталог для этого шага: {os.path.abspath(processing_dir_step1)}")

# Функция для вывода дат изменения файлов
def print_file_modification_dates(directory):
    print(f"Даты последнего изменения файлов в '{directory}':")
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        if os.path.isfile(item_path): # Убедимся, что это файл
            mod_time_timestamp = os.path.getmtime(item_path)
            mod_time_datetime = datetime.datetime.fromtimestamp(mod_time_timestamp)
            print(f" - {item}: {mod_time_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

print("\nДаты изменения файлов ДО вызова метода:")
print_file_modification_dates(processing_dir_step1)

# Вызов метода
print("\nВызов date_of_change_replacement...")
# Сохраним текущее время ДО вызова, чтобы примерно понимать, какое время будет установлено
time_before_call = datetime.datetime.now()
processor.date_of_change_replacement(source_dir=processing_dir_step1)
print("Метод date_of_change_replacement завершен.")

print("\nДаты изменения файлов ПОСЛЕ вызова метода:")
print_file_modification_dates(processing_dir_step1)
print(f"(Ожидаемое время изменения примерно: {time_before_call.strftime('%Y-%m-%d %H:%M:%S')} или чуть позже)")


## 3. Группировка по частоте дискретизации и сети

Метод `grouping_by_sampling_rate_and_network` анализирует файлы `.cfg` в указанном исходном каталоге, извлекает из них информацию о частоте сети (f_network) и частоте дискретизации (f_rate). Затем он создает подкаталоги в этом же исходном каталоге, именуя их по формату `f_network = [значение] and f_rate = [значение]`, и перемещает соответствующие файлы `.cfg` и их парные `.dat` файлы в эти подкаталоги.

Это помогает организовать осциллограммы по их техническим характеристикам.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` и `.dat` для группировки. Файлы будут перемещены из этого каталога в созданные в нем же подкаталоги.
*   `threshold` (float, опционально, по умолчанию 0.1): Порог для определения отклонения частоты от целого числа как ошибки измерения. Используется в `extract_frequencies`.
*   `isPrintMessege` (bool, опционально, по умолчанию `False`): Печатать ли сообщения, если частоты не найдены. Используется в `extract_frequencies`.

**Побочные эффекты:**
*   Создает новые подкаталоги внутри `source_dir`.
*   Перемещает файлы `.cfg` и `.dat` из `source_dir` в созданные подкаталоги.

In [ ]:
# --- 3. Демонстрация grouping_by_sampling_rate_and_network ---

# Для этого шага создадим новый каталог, чтобы наглядно показать создание подпапок.
# Скопируем туда несколько "обработанных" (переименованных хешами) файлов из processing_dir_step1,
# если он существует, или создадим новые фиктивные файлы с разными частотами.

processing_dir_step3 = os.path.join(base_source_cfg_dir, "step3_grouping")
if os.path.exists(processing_dir_step3):
    shutil.rmtree(processing_dir_step3)
os.makedirs(processing_dir_step3)

print(f"Каталог для этого шага: {os.path.abspath(processing_dir_step3)}")

# Пример 1: Частота сети 50 Гц, частота дискретизации 1000 Гц
cfg_content_g1 = ",,REC_VERS=1\n2,1A,1D\nAnalogS1,PhA,,,1,1,0,-32768,32767,V,S\nDigitalS1,0\n50\n1\n1000,1\n01/01/0001,00:00:00.000\n01/01/0001,00:00:01.000\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step3, "file_50_1000", content_cfg=cfg_content_g1, content_dat="dat_50_1000")

# Пример 2: Частота сети 50 Гц, частота дискретизации 2500 Гц
cfg_content_g2 = ",,REC_VERS=1\n2,1A,1D\nSignalU,PhB,,,1,1,0,-32768,32767,V,S\nSignalD,0\n50\n1\n2500,1\n01/01/0001,00:00:00.000\n01/01/0001,00:00:01.000\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step3, "file_50_2500", content_cfg=cfg_content_g2, content_dat="dat_50_2500")

# Пример 3: Частота сети 60 Гц, частота дискретизации 1200 Гц
cfg_content_g3 = ",,REC_VERS=1\n2,1A,1D\nVoltageA,PhA,,,1,1,0,-32768,32767,V,S\nStatusX,0\n60\n1\n1200,1\n01/01/0001,00:00:00.000\n01/01/0001,00:00:01.000\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step3, "file_60_1200", content_cfg=cfg_content_g3, content_dat="dat_60_1200")

# Пример 4: Файл с некорректной или отсутствующей информацией о частотах (останется в корне)
cfg_content_g4 = ",,REC_VERS=1\n2,1A,1D\nBadSignal,PhC,,,1,1,0,-32768,32767,V,S\nNoFreqInfo,0\n0\n1\n0,1\n01/01/0001,00:00:00.000\n01/01/0001,00:00:01.000\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step3, "file_bad_freq", content_cfg=cfg_content_g4, content_dat="dat_bad_freq")


print(f"\nИсходные файлы в '{processing_dir_step3}':")
for f_name in os.listdir(processing_dir_step3):
    print(f" - {f_name}")

# Вызов метода
print("\nВызов grouping_by_sampling_rate_and_network...")
processor.grouping_by_sampling_rate_and_network(source_dir=processing_dir_step3, isPrintMessege=True) # isPrintMessege=True для отладки
print("Метод grouping_by_sampling_rate_and_network завершен.")

print(f"\nСтруктура каталога '{processing_dir_step3}' после группировки:")
for root_dir, sub_dirs, files_in_dir in os.walk(processing_dir_step3):
    # Отступ для отображения иерархии
    level = root_dir.replace(processing_dir_step3, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root_dir)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files_in_dir:
        print(f"{sub_indent}{f_name}")


## 4. Сбор всех уникальных имен аналоговых и цифровых сигналов

На этом этапе используются два метода для сбора всех уникальных имен сигналов из файлов `.cfg` в указанном каталоге:
*   `find_all_name_analog_signals`: Собирает имена аналоговых сигналов.
*   `find_all_name_digital_signals`: Собирает имена цифровых (дискретных) сигналов.

Каждый из этих методов создает CSV-файл в том же каталоге (`source_dir`), содержащий два столбца: `Key` (имя сигнала) и `Value` (частота встречаемости этого имени во всех обработанных файлах), а также столбец `universal_code` для последующего заполнения стандартизированным именем. Файлы называются `sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv` соответственно.

**Параметры (для обоих методов):**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` для анализа. CSV-файлы с результатами будут сохранены в этот же каталог.

**Побочные эффекты:**
*   Создает CSV-файл (`sorted_analog_signals_name.csv` или `sorted_digital_signals_name.csv`) в `source_dir`.

In [ ]:
# --- 4. Демонстрация find_all_name_analog_signals и find_all_name_digital_signals ---

# Для этого шага создадим новый каталог с несколькими файлами .cfg,
# которые могут содержать как одинаковые, так и разные имена сигналов.
# Мы будем использовать этот каталог как source_dir для обоих методов.

processing_dir_step4 = os.path.join(base_source_cfg_dir, "step4_signal_names")
if os.path.exists(processing_dir_step4):
    shutil.rmtree(processing_dir_step4)
os.makedirs(processing_dir_step4)

print(f"Каталог для этого шага: {os.path.abspath(processing_dir_step4)}")

# Файл 1
cfg_s1 = ",,REC\n4,2A,2D\nUa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,V,S\nIa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,A,S\nRelay_Trip_A,0\nRelay_Start_B,0\n50\n1\n1000,1\n01/01/0001,00:00:00\n01/01/0001,00:00:01\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step4, "cfg_file_A", content_cfg=cfg_s1)

# Файл 2
cfg_s2 = ",,REC\n4,2A,2D\nUa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,V,S\nUb_System1 | phase:B,PhB,,,1,1,0,-32768,32767,V,S\nRelay_Trip_A,0\nDevice_Status,0\n50\n1\n1000,1\n01/01/0001,00:00:00\n01/01/0001,00:00:01\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step4, "cfg_file_B", content_cfg=cfg_s2)

# Файл 3 (с другими именами и одним повторяющимся аналоговым)
cfg_s3 = ",,REC\n3,2A,1D\nVoltage_X1,PhX,,,1,1,0,-32768,32767,kV,S\nUa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,V,S\nBreaker_Open,0\n50\n1\n2000,1\n01/01/0001,00:00:00\n01/01/0001,00:00:01\nDAT"
create_dummy_cfg_dat_pair(processing_dir_step4, "cfg_file_C", content_cfg=cfg_s3)

print(f"\nИсходные файлы в '{processing_dir_step4}':")
for f_name in os.listdir(processing_dir_step4):
    if f_name.endswith(".cfg"):
        print(f" - {f_name}")

# Вызов find_all_name_analog_signals
print("\nВызов find_all_name_analog_signals...")
processor.find_all_name_analog_signals(source_dir=processing_dir_step4)
analog_csv_path = os.path.join(processing_dir_step4, "sorted_analog_signals_name.csv")
print(f"Метод find_all_name_analog_signals завершен. Результаты должны быть в: {analog_csv_path}")

if os.path.exists(analog_csv_path):
    print(f"Содержимое '{analog_csv_path}' (первые несколько строк):")
    try:
        with open(analog_csv_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i < 5: print(f"  {line.strip()}")
                else: break
            if i >=4: print("  ...")
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")
else:
    print(f"  Файл '{analog_csv_path}' не найден.")

# Вызов find_all_name_digital_signals
print("\nВызов find_all_name_digital_signals...")
processor.find_all_name_digital_signals(source_dir=processing_dir_step4)
digital_csv_path = os.path.join(processing_dir_step4, "sorted_digital_signals_name.csv")
print(f"Метод find_all_name_digital_signals завершен. Результаты должны быть в: {digital_csv_path}")

if os.path.exists(digital_csv_path):
    print(f"Содержимое '{digital_csv_path}' (первые несколько строк):")
    try:
        with open(digital_csv_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i < 5: print(f"  {line.strip()}")
                else: break
            if i >=4: print("  ...")
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")
else:
    print(f"  Файл '{digital_csv_path}' не найден.")


### Важное примечание: Ручная стандартизация имен сигналов

После того как вы собрали все уникальные имена аналоговых и цифровых сигналов с помощью `find_all_name_analog_signals` и `find_all_name_digital_signals`, следующим критически важным шагом является **ручная стандартизация этих имен**.

**Процесс включает в себя:**
1.  **Анализ CSV-файлов**: Внимательно изучите сгенерированные файлы `sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv`. Обратите внимание на разнообразие имен, возможные опечатки, разные стили именования одного и того же физического сигнала.
2.  **Создание словаря замен**: Для каждого уникального имени сигнала (столбец `Key`) необходимо определить его стандартизированное, "универсальное" имя. Это универсальное имя вносится в столбец `universal_code` в этих CSV-файлах.
    *   Если для какого-то имени не удается подобрать универсальный код или оно должно быть проигнорировано, можно оставить значение по умолчанию (`-` или `?`).
3.  **Использование стандартных имен**: В вашем проекте могут существовать предопределенные стандартные имена или правила их формирования (например, `U | BusBar-1 | phase:A`). Постарайтесь придерживаться этих стандартов. В кодовой базе проекта (возможно, не в этом классе) могут быть генераторы или списки таких стандартных имен, которые могут помочь в этом процессе.
4.  **Сохранение CSV**: После заполнения столбца `universal_code` эти CSV-файлы будут использоваться методами `rename_analog_signals` и `rename_digital_signals` для фактического переименования сигналов в файлах `.cfg`.

**Пример строки в `sorted_analog_signals_name.csv` после ручной обработки:**
```csv
Key,universal_code,Value
"Ua_System1 | phase:A","U | System-1 | phase:A",3
"Ia_System1 | phase:A","I | System-1 | phase:A",1
"Voltage_X1","U | SubstationX-Line1 | phase:A",1
...
```
Этот этап требует внимательности и хорошего понимания структуры данных и стандартов именования в вашем проекте, так как от него зависит качество и согласованность данных для последующего анализа и обучения моделей.

## 5. Переименование аналоговых и цифровых сигналов

После того как CSV-файлы с уникальными именами сигналов (`sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv`) были созданы и столбец `universal_code` в них был вручную заполнен стандартизированными именами, можно приступать к фактическому переименованию сигналов в файлах `.cfg`.

Для этого используются два метода:
*   `rename_analog_signals`: Переименовывает аналоговые сигналы.
*   `rename_digital_signals`: Переименовывает цифровые (дискретные) сигналы.

Оба метода читают соответствующий CSV-файл, создают карту замен (старое имя -> универсальное имя) и применяют эти замены к именам сигналов в каждом файле `.cfg` в указанном каталоге.

**Параметры (для обоих методов):**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg`, в которых нужно переименовать сигналы.
*   `csv_dir` (str): Путь к CSV-файлу, содержащему сопоставление старых имен с универсальными кодами (например, `sorted_analog_signals_name.csv`).
*   `encoding` (str, опционально, по умолчанию 'utf8'): Кодировка CSV-файла.
*   `delimiter` (str, опционально, по умолчанию ','): Разделитель в CSV-файле.

**Побочные эффекты:**
*   Изменяет имена сигналов непосредственно в файлах `.cfg` в каталоге `source_dir`.

In [ ]:
# ПРОВЕРИТЬ!
# В конце проверить ещё эти сигналы, есть ли они? И если нет, то добавить описание
# extract_frequencies
# rename_one_signals
# combining_databases_of_unique_codes
# combining_json_hash_table


# --- 5. Демонстрация rename_analog_signals и rename_digital_signals ---

# Мы будем работать с копией файлов из предыдущего шага (step4_signal_names),
# чтобы исходные файлы для сбора имен остались неизменными для повторных запусков.
processing_dir_step5 = os.path.join(base_source_cfg_dir, "step5_rename_signals")
if os.path.exists(processing_dir_step5):
    shutil.rmtree(processing_dir_step5)

# Копируем файлы из step4 в step5
source_for_copy_step4 = os.path.join(base_source_cfg_dir, "step4_signal_names")
if os.path.exists(source_for_copy_step4) and os.listdir(source_for_copy_step4):
    shutil.copytree(source_for_copy_step4, processing_dir_step5)
    print(f"Скопированы файлы из '{source_for_copy_step4}' в '{processing_dir_step5}' для этого шага.")
else:
    # Если step4 не был запущен или пуст, создадим фиктивные файлы снова
    os.makedirs(processing_dir_step5, exist_ok=True)
    cfg_s1_r = ",,REC\n4,2A,2D\nUa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,V,S\nIa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,A,S\nRelay_Trip_A,0\nRelay_Start_B,0\n50\n1\n1000,1\n01/01/0001,00:00:00\n01/01/0001,00:00:01\nDAT"
    create_dummy_cfg_dat_pair(processing_dir_step5, "cfg_file_A_rename.cfg", content_cfg=cfg_s1_r)
    cfg_s2_r = ",,REC\n4,2A,2D\nUa_System1 | phase:A,PhA,,,1,1,0,-32768,32767,V,S\nUb_System1 | phase:B,PhB,,,1,1,0,-32768,32767,V,S\nRelay_Trip_A,0\nDevice_Status,0\n50\n1\n1000,1\n01/01/0001,00:00:00\n01/01/0001,00:00:01\nDAT"
    create_dummy_cfg_dat_pair(processing_dir_step5, "cfg_file_B_rename.cfg", content_cfg=cfg_s2_r)
    print(f"Созданы фиктивные файлы в '{processing_dir_step5}', так как '{source_for_copy_step4}' не найден или пуст.")


print(f"Каталог для этого шага: {os.path.abspath(processing_dir_step5)}")

# --- Подготовка фиктивных CSV-файлов для переименования ---
# Эти CSV-файлы имитируют результат ручной стандартизации из шага 4.

# Фиктивный CSV для аналоговых сигналов
dummy_analog_rename_csv_path = os.path.join(processing_dir_step5, "dummy_analog_rename_map.csv")
analog_rename_data = [
    ["Key", "universal_code", "Value"],
    ["Ua_System1 | phase:A", "U | System-1 | phase:A", "3"], # '3' - фиктивная частота встречаемости
    ["Ia_System1 | phase:A", "I | System-1 | phase:A", "1"],
    ["Ub_System1 | phase:B", "U | System-1 | phase:B", "1"],
    ["Voltage_X1", "U | SubstationX-Line1 | phase:A", "1"] # Это имя было в cfg_file_C
]
with open(dummy_analog_rename_csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(analog_rename_data)
print(f"Создан фиктивный CSV для переименования аналоговых сигналов: {dummy_analog_rename_csv_path}")

# Фиктивный CSV для цифровых сигналов
dummy_digital_rename_csv_path = os.path.join(processing_dir_step5, "dummy_digital_rename_map.csv")
digital_rename_data = [
    ["Key", "universal_code", "Value"],
    ["Relay_Trip_A", "Protection | Relay | Trip | phase:A", "2"],
    ["Relay_Start_B", "Protection | Relay | Start | phase:B", "1"],
    ["Device_Status", "Status | Device | General", "1"],
    ["Breaker_Open", "Switch | Breaker | Open", "1"] # Это имя было в cfg_file_C
]
with open(dummy_digital_rename_csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(digital_rename_data)
print(f"Создан фиктивный CSV для переименования цифровых сигналов: {dummy_digital_rename_csv_path}")


# Вывод содержимого одного из CFG файлов ДО переименования
example_cfg_before_path = os.path.join(processing_dir_step5, "cfg_file_A_rename.cfg") # Имя фиктивного файла, если step4 не был запущен
if not os.path.exists(example_cfg_before_path) and os.path.exists(os.path.join(processing_dir_step5, "cfg_file_A.cfg")): # Если скопировали из step4
    example_cfg_before_path = os.path.join(processing_dir_step5, "cfg_file_A.cfg")

if os.path.exists(example_cfg_before_path):
    print(f"\nСодержимое '{example_cfg_before_path}' ДО переименования (первые несколько строк сигналов):")
    try:
        with open(example_cfg_before_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if 1 < i < 7 : print(f"  {line.strip()}") # Показать строки с именами сигналов
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")


# Вызов rename_analog_signals
print("\nВызов rename_analog_signals...")
processor.rename_analog_signals(source_dir=processing_dir_step5, csv_dir=dummy_analog_rename_csv_path)
print("Метод rename_analog_signals завершен.")

# Вызов rename_digital_signals
print("\nВызов rename_digital_signals...")
processor.rename_digital_signals(source_dir=processing_dir_step5, csv_dir=dummy_digital_rename_csv_path)
print("Метод rename_digital_signals завершен.")

# Вывод содержимого того же CFG файла ПОСЛЕ переименования
if os.path.exists(example_cfg_before_path): # Имя файла не меняется
    print(f"\nСодержимое '{example_cfg_before_path}' ПОСЛЕ переименования (первые несколько строк сигналов):")
    try:
        with open(example_cfg_before_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if 1 < i < 7 : print(f"  {line.strip()}")
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")
